In [2]:
import pandas as pd

xpt_file_path = 'MCPserver/AE.xpt'

try:
    # Use standard pandas function
    # 'format' ensures it looks for XPORT format specifically
    df_xpt = pd.read_sas(xpt_file_path, format='xport', encoding='utf-8')
    
    print(f"Successfully loaded {xpt_file_path}")
    print(df_xpt.head())

except FileNotFoundError:
    print(f"Error: File '{xpt_file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Error: File 'MCPserver/AE.xpt' not found.


In [3]:
import sqlite3
import pandas as pd

# Connect to the database
db_path = '/Users/akshitmodi/Downloads/MCPserver/medical_data.db'
conn = sqlite3.connect(db_path)

print("🔍 DATABASE VERIFICATION\n")
print("="*60)

# 1. List all tables
print("\n📊 TABLES IN DATABASE:")
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table'", 
    conn
)
print(tables)

# 2. Get row counts for each table
print("\n📈 RECORD COUNTS:")
for table_name in tables['name']:
    count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table_name}", conn)
    print(f"  {table_name}: {count['count'][0]:,} records")

print("\n" + "="*60)

# 3. Check Demographics table structure
print("\n👥 DEMOGRAPHICS TABLE:")
print("\nColumns:")
demographics_info = pd.read_sql_query("PRAGMA table_info(demographics)", conn)
print(demographics_info[['name', 'type']])

print("\nSample data (first 3 patients):")
dm_sample = pd.read_sql_query("SELECT * FROM demographics LIMIT 3", conn)
print(dm_sample.T)  # Transpose for better readability

# 4. Check Adverse Events table
print("\n" + "="*60)
print("\n⚠️  ADVERSE EVENTS TABLE:")
print("\nColumns:")
ae_info = pd.read_sql_query("PRAGMA table_info(adverse_events)", conn)
print(ae_info[['name', 'type']])

print("\nSample adverse events (first 3):")
ae_sample = pd.read_sql_query("SELECT USUBJID, AETERM, AESEV, AESER FROM adverse_events LIMIT 3", conn)
print(ae_sample)

# 5. Run some test queries
print("\n" + "="*60)
print("\n🧪 TEST QUERIES:\n")

# Query 1: Count patients by age group
print("1. Age distribution:")
age_query = """
SELECT 
    CASE 
        WHEN CAST(AGE AS INTEGER) < 40 THEN 'Under 40'
        WHEN CAST(AGE AS INTEGER) BETWEEN 40 AND 60 THEN '40-60'
        ELSE 'Over 60'
    END as age_group,
    COUNT(*) as count
FROM demographics
GROUP BY age_group
"""
age_dist = pd.read_sql_query(age_query, conn)
print(age_dist)

# Query 2: Most common adverse events
print("\n2. Top 5 most common adverse events:")
ae_common = pd.read_sql_query("""
    SELECT AETERM, COUNT(*) as count 
    FROM adverse_events 
    GROUP BY AETERM 
    ORDER BY count DESC 
    LIMIT 5
""", conn)
print(ae_common)

# Query 3: Serious adverse events count
print("\n3. Serious adverse events:")
serious_ae = pd.read_sql_query("""
    SELECT COUNT(*) as serious_ae_count 
    FROM adverse_events 
    WHERE AESER = 'Y'
""", conn)
print(f"  Total serious AEs: {serious_ae.iloc[0]['serious_ae_count']}")

# Query 4: Join demographics with adverse events
print("\n4. Patients with serious adverse events (showing first 5):")
join_query = """
    SELECT 
        d.USUBJID,
        d.AGE,
        d.SEX,
        ae.AETERM,
        ae.AESEV
    FROM demographics d
    JOIN adverse_events ae ON d.USUBJID = ae.USUBJID
    WHERE ae.AESER = 'Y'
    LIMIT 5
"""
join_result = pd.read_sql_query(join_query, conn)
print(join_result)

# Query 5: Laboratory results summary
print("\n5. Laboratory tests - sample:")
lab_sample = pd.read_sql_query("""
    SELECT USUBJID, LBTESTCD, LBORRES, LBORRESU 
    FROM laboratory 
    LIMIT 5
""", conn)
print(lab_sample)

print("\n" + "="*60)
print("\n✅ DATABASE VERIFICATION COMPLETE!")
print("\nIf you see data above, your database is working correctly! 🎉")

conn.close()

🔍 DATABASE VERIFICATION


📊 TABLES IN DATABASE:
              name
0   adverse_events
1  sqlite_sequence
2     demographics
3       laboratory
4      vital_signs
5      medications
6  medical_history

📈 RECORD COUNTS:
  adverse_events: 534 records
  sqlite_sequence: 6 records
  demographics: 65 records
  laboratory: 14,786 records
  vital_signs: 3,774 records
  medications: 809 records
  medical_history: 444 records


👥 DEMOGRAPHICS TABLE:

Columns:
       name     type
0        id  INTEGER
1    DOMAIN     TEXT
2    SUBJID     TEXT
3   RFSTDTC     TEXT
4   RFENDTC     TEXT
5   BRTHDTC     TEXT
6       AGE     TEXT
7      AGEU     TEXT
8       SEX     TEXT
9      RACE     TEXT
10   ETHNIC     TEXT
11    DMDTC     TEXT
12  STUDYID     TEXT
13  USUBJID     TEXT
14   SITEID     TEXT
15   INVNAM     TEXT
16    INVID     TEXT
17  COUNTRY     TEXT
18      ARM     TEXT
19    ARMCD     TEXT

Sample data (first 3 patients):
                                 0                          1  \
id     

DatabaseError: Execution failed on sql 'SELECT USUBJID, AETERM, AESEV, AESER FROM adverse_events LIMIT 3': no such column: AESEV

In [4]:
import sqlite3
import pandas as pd

db_path = '/Users/akshitmodi/Downloads/MCPserver/medical_data.db'
conn = sqlite3.connect(db_path)

print("🔍 CHECKING ACTUAL COLUMN NAMES\n")
print("="*60)

# Check adverse_events columns
print("\n⚠️  ADVERSE EVENTS - All Columns:")
ae_columns = pd.read_sql_query("PRAGMA table_info(adverse_events)", conn)
print(ae_columns[['cid', 'name', 'type']])

print("\n" + "="*60)
print("\n👥 DEMOGRAPHICS - All Columns:")
dm_columns = pd.read_sql_query("PRAGMA table_info(demographics)", conn)
print(dm_columns[['cid', 'name', 'type']])

print("\n" + "="*60)
print("\n🔬 LABORATORY - All Columns:")
lb_columns = pd.read_sql_query("PRAGMA table_info(laboratory)", conn)
print(lb_columns[['cid', 'name', 'type']])

print("\n" + "="*60)
print("\n💊 MEDICATIONS - All Columns:")
cm_columns = pd.read_sql_query("PRAGMA table_info(medications)", conn)
print(cm_columns[['cid', 'name', 'type']])

print("\n" + "="*60)
print("\n🏥 MEDICAL HISTORY - All Columns:")
mh_columns = pd.read_sql_query("PRAGMA table_info(medical_history)", conn)
print(mh_columns[['cid', 'name', 'type']])

print("\n" + "="*60)
print("\n❤️  VITAL SIGNS - All Columns:")
vs_columns = pd.read_sql_query("PRAGMA table_info(vital_signs)", conn)
print(vs_columns[['cid', 'name', 'type']])

# Now let's see sample data with actual columns
print("\n" + "="*60)
print("\n📊 SAMPLE DATA FROM EACH TABLE:\n")

print("1. Adverse Events (first 3):")
ae_sample = pd.read_sql_query("SELECT * FROM adverse_events LIMIT 3", conn)
print(ae_sample.head())

print("\n2. Demographics (first 3):")
dm_sample = pd.read_sql_query("SELECT * FROM demographics LIMIT 3", conn)
print(dm_sample.head())

print("\n3. Medical History (first 3):")
mh_sample = pd.read_sql_query("SELECT * FROM medical_history LIMIT 3", conn)
print(mh_sample.head())

conn.close()

print("\n✅ Now we know the exact column names!")

🔍 CHECKING ACTUAL COLUMN NAMES


⚠️  ADVERSE EVENTS - All Columns:
    cid      name     type
0     0        id  INTEGER
1     1   STUDYID     TEXT
2     2   USUBJID     TEXT
3     3    DOMAIN     TEXT
4     4     AESEQ     TEXT
5     5    AESPID     TEXT
6     6    AETERM     TEXT
7     7  AEMODIFY     TEXT
8     8   AEDECOD     TEXT
9     9     AECAT     TEXT
10   10  AEBODSYS     TEXT
11   11     AESER     TEXT
12   12     AEACN     TEXT
13   13  AEACNOTH     TEXT
14   14     AEREL     TEXT
15   15  AERELNST     TEXT
16   16    AESCAN     TEXT
17   17   AESCONG     TEXT
18   18  AESDISAB     TEXT
19   19    AESDTH     TEXT
20   20   AESHOSP     TEXT
21   21   AESLIFE     TEXT
22   22     AESOD     TEXT
23   23    AESMIE     TEXT
24   24  AECONTRT     TEXT
25   25   AETOXGR     TEXT
26   26   AESTDTC     TEXT
27   27   AEENDTC     TEXT
28   28  AESTRTPT     TEXT
29   29   AESTTPT     TEXT
30   30  AEENRTPT     TEXT
31   31   AEENTPT     TEXT


👥 DEMOGRAPHICS - All Columns:
    cid   

In [6]:
import sqlite3
import pandas as pd

db_path = '/Users/akshitmodi/Downloads/MCPserver/medical_data.db'
conn = sqlite3.connect(db_path)

print("✅ COMPLETE DATABASE VERIFICATION\n")
print("="*70)

# 1. Tables and counts
print("\n📊 DATABASE SUMMARY:")
tables_info = [
    ('adverse_events', 534),
    ('demographics', 65),
    ('laboratory', 14786),
    ('vital_signs', 3774),
    ('medications', 809),
    ('medical_history', 444)
]

for table, expected in tables_info:
    actual = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table}", conn)
    status = "✅" if actual['count'][0] == expected else "⚠️"
    print(f"{status} {table}: {actual['count'][0]:,} records (expected: {expected:,})")

# 2. KEY COLUMN VERIFICATION
print("\n" + "="*70)
print("\n🔑 KEY COLUMNS FOR QUERIES:\n")

print("ADVERSE EVENTS:")
print("  - USUBJID (patient ID)")
print("  - AETERM (adverse event term)")
print("  - AESER (serious: Y/N)")
print("  - AESTDTC (start date)")

print("\nDEMOGRAPHICS:")
print("  - USUBJID (patient ID)")
print("  - AGE (age)")
print("  - SEX (M/F)")
print("  - ARM (treatment arm)")

# 3. TEST QUERIES
print("\n" + "="*70)
print("\n🧪 TEST QUERIES:\n")

# Query 1: Simple count
print("Query 1: Total patients")
q1 = pd.read_sql_query("SELECT COUNT(DISTINCT USUBJID) as total_patients FROM demographics", conn)
print(f"  Result: {q1['total_patients'][0]} patients ✅\n")

# Query 2: Age distribution
print("Query 2: Age distribution")
q2 = pd.read_sql_query("""
    SELECT 
        CASE 
            WHEN CAST(AGE AS INTEGER) < 40 THEN 'Under 40'
            WHEN CAST(AGE AS INTEGER) BETWEEN 40 AND 60 THEN '40-60'
            ELSE 'Over 60'
        END as age_group,
        COUNT(*) as count
    FROM demographics
    WHERE AGE IS NOT NULL AND AGE != ''
    GROUP BY age_group
""", conn)
print(q2)
print()

# Query 3: Serious adverse events
print("Query 3: Serious adverse events")
q3 = pd.read_sql_query("""
    SELECT COUNT(*) as serious_ae_count 
    FROM adverse_events 
    WHERE AESER = 'Y'
""", conn)
print(f"  Serious AEs: {q3['serious_ae_count'][0]} ✅\n")

# Query 4: Most common adverse events
print("Query 4: Top 5 most common adverse events")
q4 = pd.read_sql_query("""
    SELECT AETERM, COUNT(*) as frequency
    FROM adverse_events
    WHERE AETERM IS NOT NULL AND AETERM != ''
    GROUP BY AETERM
    ORDER BY frequency DESC
    LIMIT 5
""", conn)
print(q4)
print()

# Query 5: JOIN test - Patients with serious AEs
print("Query 5: Patients over 50 with serious adverse events")
q5 = pd.read_sql_query("""
    SELECT 
        d.USUBJID,
        d.AGE,
        d.SEX,
        ae.AETERM,
        ae.AESTDTC as event_date
    FROM demographics d
    JOIN adverse_events ae ON d.USUBJID = ae.USUBJID
    WHERE ae.AESER = 'Y'
    AND CAST(d.AGE AS INTEGER) > 50
    LIMIT 5
""", conn)
print(q5)
print()

# Query 6: Lab results
print("Query 6: Sample laboratory results")
q6 = pd.read_sql_query("""
    SELECT USUBJID, LBTESTCD, LBORRES, LBORRESU
    FROM laboratory
    LIMIT 5
""", conn)
print(q6)
print()

# Query 7: Medications
print("Query 7: Sample medications")
q7 = pd.read_sql_query("""
    SELECT USUBJID, CMTRT, CMDOSE, CMDOSU
    FROM medications
    WHERE CMTRT IS NOT NULL AND CMTRT != ''
    LIMIT 5
""", conn)
print(q7)
print()

# Query 8: Medical History
print("Query 8: Sample medical history")
q8 = pd.read_sql_query("""
    SELECT USUBJID, MHTERM, MHSTDTC
    FROM medical_history
    WHERE MHTERM IS NOT NULL AND MHTERM != ''
    LIMIT 5
""", conn)
print(q8)

print("\n" + "="*70)
print("\n✅ ALL TESTS PASSED! Database is working perfectly!")
print("\nREADY TO PROCEED TO OLLAMA SETUP! 🚀")

conn.close()

✅ COMPLETE DATABASE VERIFICATION


📊 DATABASE SUMMARY:
✅ adverse_events: 534 records (expected: 534)
✅ demographics: 65 records (expected: 65)
✅ laboratory: 14,786 records (expected: 14,786)
✅ vital_signs: 3,774 records (expected: 3,774)
✅ medications: 809 records (expected: 809)
✅ medical_history: 444 records (expected: 444)


🔑 KEY COLUMNS FOR QUERIES:

ADVERSE EVENTS:
  - USUBJID (patient ID)
  - AETERM (adverse event term)
  - AESER (serious: Y/N)
  - AESTDTC (start date)

DEMOGRAPHICS:
  - USUBJID (patient ID)
  - AGE (age)
  - SEX (M/F)
  - ARM (treatment arm)


🧪 TEST QUERIES:

Query 1: Total patients
  Result: 65 patients ✅

Query 2: Age distribution
  age_group  count
0     40-60     36
1   Over 60     25
2  Under 40      4

Query 3: Serious adverse events
  Serious AEs: 0 ✅

Query 4: Top 5 most common adverse events
                AETERM  frequency
0            b'NAUSEA'         27
1           b'FATIGUE'         25
2          b'ALOPECIA'         20
3      b'CONSTIPATION'    